In [ ]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

#gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
data_dir = "./Data/train"
csv_path = "./Data/trainLabels.csv"

df = pd.read_csv(csv_path)

class_names = sorted(df['label'].unique())
class_map = {name: i for i, name in enumerate(class_names)}


train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
print(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")


In [ ]:
image_id = 1009
print(f"Label for image {image_id}: {df.loc[df['id'] == image_id, 'label'].values[0]}")

In [ ]:
class CIFARDataset(Dataset):
    def __init__(self, dataframe, data_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_id = row['id']
        label_str = row['label']
        
        img_path = os.path.join(self.data_dir, f"{image_id}.png")
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        label = class_map[label_str]
        return image, label

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

train_dataset = CIFARDataset(train_df, data_dir, transform=transform)
val_dataset = CIFARDataset(val_df, data_dir, transform=transform)

batch_size = 64  # tunable
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Ready!")


In [ ]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

num_classes = len(class_names)
model = CNNModel(num_classes).to(device)
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

patience = 3
best_val_loss = float('inf')
epochs_no_improve = 0

num_epochs = 15
train_losses, val_losses = [], []
train_accs, val_accs = [], []

In [ ]:
writer = SummaryWriter(log_dir='Logs/no_brownies')
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    train_epoch_loss = running_loss / len(train_loader.dataset)
    train_epoch_acc = 100.0 * correct / total
    
    model.eval()
    val_running_loss = 0.0
    val_correct, val_total = 0, 0
    
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            
            val_running_loss += val_loss.item() * val_images.size(0)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == val_labels).sum().item()
            val_total += val_labels.size(0)
    
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_acc = 100.0 * val_correct / val_total
    
    train_losses.append(train_epoch_loss)
    val_losses.append(val_epoch_loss)
    train_accs.append(train_epoch_acc)
    val_accs.append(val_epoch_acc)
    
    writer.add_scalar('Loss/Train', train_epoch_loss, epoch)
    writer.add_scalar('Loss/Val', val_epoch_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_epoch_acc, epoch)
    writer.add_scalar('Accuracy/Val', val_epoch_acc, epoch)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.2f}% "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.2f}%")
    
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered!")
            break

writer.close()

In [9]:

model.load_state_dict(torch.load("best_model.pth"))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted')
rec = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Validation Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.3f}, Recall: {rec:.3f}, F1-score: {f1:.3f}")

print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

Validation Accuracy: 77.95%
Precision: 0.783, Recall: 0.779, F1-score: 0.778

Classification Report:
              precision    recall  f1-score   support

    airplane       0.80      0.83      0.82      1000
  automobile       0.85      0.91      0.88      1000
        bird       0.80      0.59      0.68      1000
         cat       0.60      0.59      0.60      1000
        deer       0.71      0.80      0.75      1000
         dog       0.68      0.68      0.68      1000
        frog       0.73      0.90      0.81      1000
       horse       0.82      0.81      0.82      1000
        ship       0.93      0.84      0.88      1000
       truck       0.88      0.86      0.87      1000

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000

